## Import Libraries

In [1]:
import pandas as pd
import requests
from tqdm import tqdm

## Import Data

In [2]:
df_main = pd.read_csv('data/rishi-stores.csv')              # stores dataset
df_reference = pd.read_csv('data/safegraph-raw.csv')        # reference with placekeys (condensed) 

# Process Data
Using geocoding

### Step 1
Iterate through the addresses in the reference, making: <br>
   > 1. an extra column for geocodes <br>
   > 2. a hashtable such that hash[geocode] = placekey

In [ ]:
geocode_dict = {}                                 # create empty dictionary to store geocode:placekey
API_KEY = '--INSERT API KEY HERE--'               # update API key when purchased 

for i in tqdm(range(len(df_reference))):
    address = str(df_reference.loc[i, "street_address"]) + ', ' + str(df_reference.loc[i, "region"])

    params = {
        'key': API_KEY,
        'address': address
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(base_url, params=params).json()
    
    if response['status'] == 'OK':
        geometry = response['results'][0]['geometry']
        lat = geometry['location']['lat'] 
        lon = geometry['location']['lng'] 
        geocode_address = '(' + str(lat) + ',' + str(lon) + ')'
        df_reference.loc[i, "geocode"] = geocode_coords
        geocode_dict[geocode_coords] = df_reference.loc[i, "placekey"]
    else:
        print('Errors with', address)

In [ ]:
df_reference.to_csv('./safegraph-raw-geocode', index=False)

### Step 2
Iterate through the addresses in the stores dataset: <br>
   > 1. find the geocode for each address in stores <br>
   > 2. key into geocode_dict to find corresponding placekey
   > 3. build placekey column in stores

In [ ]:
for i in range(len(df_main)):
    address = str(df_main.loc[i, "address"]) + ', ' + str(df_main.loc[i, "state_code"])
    
    params = {
        'key': API_KEY,
        'address': address
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(base_url, params=params).json()
    
    if response['status'] == 'OK':
        geometry = response['results'][0]['geometry']
        lat = geometry['location']['lat'] 
        lon = geometry['location']['lng'] 
        geocode_address = '(' + str(lat) + ',' + str(lon) + ')'
        df_main.loc[i, "placekey"] = geocode_dict[geocode_address]
    else:
        print('Errors with', address)

### Step 3
Export new dataset as csv file

In [ ]:
df_main.to_csv('./data/rishi-stores-placekeys.csv', index=False)